# Step 0. 필요한 라이브러리 및 모듈 불러오기, 파일 다운로드!!!

이번 프로젝트는 Word-Level 번역기를 만드는 과정을 담은 해당 블로그를 상당 부분 참고하였다. 처음 보는 형태의 모듈이나 기타 의문점이 드는 사항들은 주석을 달아가며 작성해 봤다!!!  
참고 링크 : https://wikidocs.net/86900

In [1]:
import os          # 개별 파일에 대한 연산 모듈
import re
import shutil      # 파일 모음에 대한 여러 가지 고수준 연산을 제공하는 기능의 모듈
import zipfile     # 압축 파일에 대한 연산 모듈

import numpy as np
import pandas as pd
import unicodedata     # 유니코드 문자에 대한 문자 속성을 정의하는 유니코드 문자 데이터베이스에 대한 액세스 제공!!
import urllib3         # 파이썬용 강력하고 사용자 친화적인 HTTP 클라이언트. gzip, deflate 및 brotli 인코딩 지원 및 기타 다양한 기능 제공!!
import tensorflow as tf

from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

print(tf.__version__)

2.8.2


처음으로 접하는 모듈이 몇 가지가 있어 간략하게 알아보자!!! 링크를 통해 해당 모듈이 어떤 기능을 하는지 위주로 챙겨보기!!  
* shutil : https://docs.python.org/ko/3/library/shutil.html  
* unicodedata : https://docs.python.org/ko/3/library/unicodedata.html  
* urlib3 : https://urllib3.readthedocs.io/en/stable/

urllib3랑 zipfile 등의 모듈을 활용해 url 상의 fra-eng.zim 파일을 내려받고 압축 파일을 풀어보는 과정을 진행하였다. 

In [2]:
http = urllib3.PoolManager()
url = 'http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:
  shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
  zip_ref.extractall(path)

실제로 앞선 과정을 밟아오면서 우리가 사용했던 그 파일을 다운받은 것이지만, 아까와는 달리 인터넷 상에서의 파일을 코랩에서 바로 다운받는 코드를 직접 작성하고 압축 파일 해제까지를 완료한 상태다. 때문에 현재 해당 데이터셋이 이제 코랩에서 정상적으로 인지할 수 있는 상태가 되었다. fra.txt에 있는 파일을 pandas를 통해 읽어들여서 한번 샘플 확인을 해보자!!

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
file_path = '/content/drive/MyDrive/AIFFEL/DATASET/fra.txt'
lines =  pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 194513


,eng,fra,cc
183290,Mary was a tomboy when she was in elementary s...,Mary était un garçon manqué quand elle était à...,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
87960,Keep your eyes on the road.,Garde les yeux sur la route !,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
181011,It seems like the cat caught the scent of a mo...,On dirait que le chat a détecté l'odeur d'une ...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
43688,It's still too early.,Il est encore trop tôt.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
177380,Why don't you start by telling us how you feel?,Pourquoi ne commences-tu pas par nous dire ce ...,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


앞선 과정과 동일하게 영어-불어 19만개의 데이터들이 담긴 데이터셋을 활용하는 것임을 재확인했다. 이번에는 프로젝트 실행 조건에 맞게끔 데이터 개수를 33000개로 한정지었다. 다음 코드를 작성하면 데이터 사용 개수를 한정지을 수 있다.

In [5]:
# 데이터에서 상위 33,000개의 샘플만 사용하도록 데이터 개수를 한정지었다.
num_samples = 33000
lines = lines[['eng', 'fra']][:33000]
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 33000


,eng,fra
15082,Just stay there.,Reste ici.
26162,This is messed up.,C'est foutu.
11366,Make it larger.,Élargissez-le.
1660,Are you in?,Es-tu partant ?
700,It's odd.,C'est bizarre.


# Step 1. 정제, 정규화, 전처리(영어, 불어 모두)

글자 단위가 아닌 단어 단위의 번역기를 만드는 것이 목표이기 때문에 글자 단위에서 신경쓰지 않았던 몇 가지 추가적인 전처리 작업이 필요하다.

단어라고 함은 띄어쓰기를 기준으로 분리하는데,  어디서부터 어디까지가 하나의 단어인지를 명확히 해야만 한다.

또한 구두점을 분리하는 작업도 필요하다. 이를 신경 쓰지 않는다면 단어 뒤에 붙어있는 !나 ?나 . 같은 구두점을 포함한 채로 토큰화가 진행되게 된다. 하지만 실제로 구두점은 어떤 단어와 붙어있는 한 단어가 아니기 때문에 분리를 해주는 것이다.

또한 영어와 달리 프랑스어의 경우는 아래 링크를 통해 알 수 있듯이 악쌍이 붙어 있는 단어들이 많이 있다. 이 악쌍 역시 제거해 줘야 나중에 번역기를 돌릴 때 이상 없이 나올 수 있을 것이기 때문에 미리 없애는 작업을 진행한다.
프랑스어의 악쌍(accents) 관련 참조 링크 : https://in2youruniverse.tistory.com/9

In [6]:
# 불어의 경우는 악센트가 붙어 있는 철자가 상당수 존재한다. 이 악센트를 삭제해야 한다.
# 예시 : 'déjà diné' -> deja dine
def to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
    if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sent):
  # 악센트 제거 함수 호출
  sent = to_ascii(sent.lower())

  # 단어와 구두점 사이에 공백 추가.
  # 띄어쓰기 단위로 토큰화를 수행해야 하기 때문에 이런 분리작업이 필요하다!!
  # ex) "I am a student." => "I am a student ."
  sent = re.sub(r"([?.!,¿])", r" \1", sent)

  # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환.
  sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

  # 다수 개의 공백을 하나의 공백으로 치환
  sent = re.sub(r"\s+", " ", sent)
  return sent

In [7]:
# 전처리 테스트
en_sent = u"I really need to go outside and get some fresh..."
fr_sent = u"As-tu déjà été diagnostiquée séropositive?"

print('전처리 전 영어 문장 :', en_sent)
print('전처리 후 영어 문장 :',preprocess_sentence(en_sent))
print('전처리 전 불어 문장 :', fr_sent)
print('전처리 후 불어 문장 :', preprocess_sentence(fr_sent))

전처리 전 영어 문장 : I really need to go outside and get some fresh...
전처리 후 영어 문장 : i really need to go outside and get some fresh . . .
전처리 전 불어 문장 : As-tu déjà été diagnostiquée séropositive?
전처리 후 불어 문장 : as tu deja ete diagnostiquee seropositive ?


전체 데이터에서 33,000개의 샘플에 대해서 전처리를 수행한다. 입력 시퀀스에는 시작을 의미하는 토큰인 `<sos>`를 추가하고, 출력 시퀀스에는 종료를 의미하는 토큰인 `<eos>`를 추가한다.

# Step 2. 디코더의 문장에 시작 토큰과 종료 토큰 삽입

글자 단위 번역기를 구현할 때와 마찬가지로 입력 시퀀스 맨 앞에는 시작을 의미하는 토큰이 필요하다.  
예를 들어 번역 문장이 Courez!이었다면 이 문자에 대해 각각 디코더의 입력 시퀀스와 레이블 시퀀스를 만들면 다음과 같다.

입력 시퀀스 : ['', 'courez', '!']
레이블 시퀀스 : ['courez','!', '']

In [8]:
def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []

  with open("fra.txt", "r") as lines:
    for i, line in enumerate(lines):
      # source 데이터와 target 데이터 분리
      src_line, tar_line, _ = line.strip().split('\t')

      # source 데이터 전처리
      src_line = [w for w in preprocess_sentence(src_line).split()]

      # target 데이터 전처리
      # 불어 입력 문장 앞에는 <sos> 토큰을, 출력 문장 뒤에는 <eos> 토큰을 첨가한다는 뜻!!
      tar_line = preprocess_sentence(tar_line)
      tar_line_in = [w for w in ("<sos> " + tar_line).split()]
      tar_line_out = [w for w in (tar_line + " <eos>").split()]

      encoder_input.append(src_line)
      decoder_input.append(tar_line_in)
      decoder_target.append(tar_line_out)

      if i == num_samples - 1:
        break

  return encoder_input, decoder_input, decoder_target

인코더가 되는 영어 문장과 디코더가 되는 문장인 불어 문장의 인덱스를 임의로 가져와 토큰화가 잘 되었는지 확인해 보자!!!

In [9]:
# 0부터 32999까지의 숫자 중 임의로 삽입하여 각 문장마다 토큰화가 잘 되었는지 확인해 보는 묘미를~~
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print('인코더의 입력 :',sents_en_in[29000:29005])
print('디코더의 입력 :',sents_fra_in[29000:29005])
print('디코더의 레이블 :',sents_fra_out[29000:29005])

인코더의 입력 : [['he', 's', 'in', 'the', 'shower', '.'], ['he', 's', 'my', 'new', 'friend', '.'], ['he', 's', 'not', 'a', 'bad', 'boy', '.'], ['he', 's', 'not', 'a', 'bad', 'guy', '.'], ['he', 's', 'not', 'all', 'there', '.']]
디코더의 입력 : [['<sos>', 'il', 'est', 'dans', 'la', 'douche', '.'], ['<sos>', 'c', 'est', 'mon', 'nouvel', 'ami', '.'], ['<sos>', 'ce', 'n', 'est', 'pas', 'un', 'mauvais', 'garcon', '.'], ['<sos>', 'ce', 'n', 'est', 'pas', 'un', 'mauvais', 'bougre', '.'], ['<sos>', 'il', 'n', 'est', 'pas', 'tout', 'a', 'fait', 'la', '.']]
디코더의 레이블 : [['il', 'est', 'dans', 'la', 'douche', '.', '<eos>'], ['c', 'est', 'mon', 'nouvel', 'ami', '.', '<eos>'], ['ce', 'n', 'est', 'pas', 'un', 'mauvais', 'garcon', '.', '<eos>'], ['ce', 'n', 'est', 'pas', 'un', 'mauvais', 'bougre', '.', '<eos>'], ['il', 'n', 'est', 'pas', 'tout', 'a', 'fait', 'la', '.', '<eos>']]


# Step 3. 케라스의 토크나이저로 텍스트를 숫자로 바꾸기

딥 러닝 모델은 각 단어를 텍스트가 아닌 숫자를 처리한다. 케라스 토크나이저를 사용해서 각 단어를 고유한 정수 형태로 바꾸는 작업이 필요하다!!

In [10]:
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input = pad_sequences(encoder_input, padding='post')

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)

decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_input = pad_sequences(decoder_input, padding='post')

decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)
decoder_target = pad_sequences(decoder_target, padding='post')

In [11]:
print('인코더의 입력의 크기(shape) :', encoder_input.shape)
print('디코더의 입력의 크기(shape) :', decoder_input.shape)
print('디코더의 출력의 크기(shape) :', decoder_target.shape)

인코더의 입력의 크기(shape) : (33000, 8)
디코더의 입력의 크기(shape) : (33000, 16)
디코더의 출력의 크기(shape) : (33000, 16)


샘플은 총 33,000개 존재하며 영어 문장의 길이는 8, 불어 문장의 길이는 16인 것을 알 수 있다. 이제 단어 집합의 크기를 정의할 차례이다.

In [12]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 불어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 4672, 불어 단어 집합의 크기 : 8153


단어 집합의 크기는 각각 영어 4672개, 불어 8153개이다. 단어로부터 정수를 얻는 딕셔너리와 정수로부터 단어를 얻는 딕셔너리를 각각 생성하도록 한다. 이들은 훈련을 마치고 예측값과 실제값을 비교하는 단계에서 사용될 것이다.

In [13]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word
tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

테스트 데이터를 분리하기 전 데이터를 섞어준다. 아무래도 실질적인 번역기의 성능을 확인해 보기 위해 무작위로 자료를 섞는 작업이라고 생각했다. 먼저 순서가 섞인 정수 시퀀스 리스트(indices)를 만들어 주고, 데이터를 섞을 때는 np.random.shuffle 함수를 활용하면 된다.

In [14]:
# 정수 시퀀스 리스트(indices) 생성
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덤 시퀀스 :', indices)

랜덤 시퀀스 : [30974 29819 32075 ...  2139 10944 17036]


In [15]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [16]:
# 임의로 샘플 출력(0부터 32999까지 중 아무 숫자나 넣어보면 됨)
encoder_input[30005]

array([  2, 839,   3,   1,   0,   0,   0,   0], dtype=int32)

In [17]:
# 임의로 샘플 출력(0부터 32999까지 중 아무 숫자나 넣어보면 됨)
decoder_input[30005]

array([  2,   4,  44,  15, 492,   1,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0], dtype=int32)

In [18]:
# 임의로 샘플 출력(0부터 32999까지 중 아무 숫자나 넣어보면 됨)
decoder_target[30005]

array([  4,  44,  15, 492,   1,   3,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0], dtype=int32)

이 때 decoder_input과 decoder_target은 데이터 구조상으로 앞에 붙은 `<sos>` 토큰과 뒤에 붙은 `<eos>` 토큰을 제외하면 동일한 정수 시퀀스를 가져야 한다. 같은 문장이기 때문에 동일한 정수 시퀀스를 가져야 될 것으로 여겨진다.

이 말은 결과를 보면 확연히 알 수 있는데, `decoder_input`의 2가 바로 `<sos>` 토큰을 의미하는 것이고, `decoder_output` 결과 내 3이 바로 `<eos>` 토큰을 의미하는 것이다. 그리고 이 두 개의 숫자를 제외한 나머지 숫자의 시퀀스를 비교해 보면 [11, 25, 77, 1822, 1]로 일치하는 것을 확인할 수 있었고 위의 개념을 직관적으로 이해할 수 있는 대목이라고 할 수 있다!!!!

이제부터는 33,000개의 데이터들 중 3,000개의 데이터를 테스트 데이터로써 사용한다!!

In [19]:
n_of_val = int(33000/11)
print('검증 데이터의 개수 :',n_of_val)

검증 데이터의 개수 : 3000


In [20]:
# 훈련 데이터 설정
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

# 테스트 데이터 설정
encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [21]:
# 훈련 데이터와 테스트 데이터의 크기 출력!!
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (30000, 8)
훈련 target 데이터의 크기 : (30000, 16)
훈련 target 레이블의 크기 : (30000, 16)
테스트 source 데이터의 크기 : (3000, 8)
테스트 target 데이터의 크기 : (3000, 16)
테스트 target 레이블의 크기 : (3000, 16)


# Step 4. 임베딩 층 사용 및 모델 구현

In [22]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

임베딩 벡터의 차원과 LSTM의 은닉 상태의 크기를 64로 설정한다.

In [23]:
embedding_dim = 64
hidden_units = 64

이제 인코더를 본격적으로 설계하고자 한다. 인코더를 주목해보면 함수형 API를 사용한다는 것 외에는 앞서 다른 실습에서 본 LSTM 설계와 크게 다르지는 않다. Masking은 패딩 토큰인 숫자 0의 경우에는 연산을 제외하는 역할을 수행한다. 인코더의 내부 상태를 디코더로 넘겨줘야 하기 때문에 return_state=True로 설정한다.

LSTM에서 state_h, state_c를 리턴받는데, 이는 각각 RNN 챕터에서 LSTM을 처음 설명할 때 언급하였던 은닉 상태와 셀 상태에 해당된다. 이 두 가지 상태를 encoder_states에 저장한다. encoder_states를 디코더에 전달하므로서 이 두 가지 상태 모두를 디코더로 전달할 것이다.

In [24]:
# 인코더 설계
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs)  # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb)   # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(hidden_units, return_state=True)  # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)  # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c]   # 인코더의 은닉 상태와 셀 상태 저장

디코더는 인코더의 마지막 은닉 상태로부터 초기 은닉 상태를 얻는다. initial_state의 인자값으로 encoder_states를 주는 코드가 이에 해당된다. 디코더도 은닉 상태, 셀 상태를 리턴하기는 하지만 훈련 과정에서는 사용하지 않는다. seq2seq의 디코더는 기본적으로 각 시점마다 다중 클래스 분류 문제를 풀고 있다. 매 시점마다 프랑스어 단어 집합의 크기의 선택지에서 단어를 1개 선택해 이를 이번 시점에서 예측한 단어로 택한다. 다중 클래스 분류 문제이므로 출력층으로 소프트맥스 함수와 손실 함수를 크로스 엔트로피 함수를 사용한다.

categorical_crossentropy를 사용하려면 레이블은 원-핫 인코딩이 된 상태여야 한다. 그런데 현재 decoder_outputs의 경우에는 원-핫 인코딩을 하지 않은 상태이다. 원-핫 인코딩을 하지 않은 상태로 정수 레이블에 대해서 다중 클래스 분류 문제를 풀고자 하는 경우에는 categorical_crossentropy가 아니라 sparse_categorical_crossentropy를 사용하면 된다.

다중 클래스 손실 함수 관련 링크 : https://crazyj.tistory.com/153

In [25]:
# 디코더 설계
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, hidden_units) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

In [26]:
# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 모델의 입력과 출력을 정의.
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

이제 모델을 훈련시킬 것이다. 128개 배치 크기로 총 50 에포크의 학습을 진행한다. 테스트 데이터를 검증 데이터로 사용해 훈련이 제대로 되고 있는지 확인해보자!!

In [27]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

Epoch 1/50
235/235 [==============================] - 26s 46ms/step - loss: 3.4086 - acc: 0.6117 - val_loss: 2.0788 - val_acc: 0.6158
Epoch 2/50
235/235 [==============================] - 9s 37ms/step - loss: 1.8969 - acc: 0.6536 - val_loss: 1.7898 - val_acc: 0.7029
Epoch 3/50
235/235 [==============================] - 9s 37ms/step - loss: 1.6885 - acc: 0.7318 - val_loss: 1.6250 - val_acc: 0.7489
Epoch 4/50
235/235 [==============================] - 9s 36ms/step - loss: 1.5451 - acc: 0.7558 - val_loss: 1.4995 - val_acc: 0.7620
Epoch 5/50
235/235 [==============================] - 8s 36ms/step - loss: 1.4285 - acc: 0.7674 - val_loss: 1.4007 - val_acc: 0.7716
Epoch 6/50
235/235 [==============================] - 9s 37ms/step - loss: 1.3273 - acc: 0.7824 - val_loss: 1.3067 - val_acc: 0.7919
Epoch 7/50
235/235 [==============================] - 9s 36ms/step - loss: 1.2343 - acc: 0.8002 - val_loss: 1.2342 - val_acc: 0.8062
Epoch 8/50
235/235 [==============================] - 9s 36ms/step -

# Step 5. 번역기 구현 &모델 평가하기

번역하는 과정은 크게 3가지 단계로 이루어진다.  

1) 번역하고자 하는 입력 문장이 인코더로 입력되어 인코더의 마지막 시점의 은닉 상태와 셀 상태를 얻는다.  
2) 인코더의 은닉 상태와 셀 상태, 그리고 토큰 `<sos>`를 디코더로 보낸다.
3) 디코더가 토큰 `<eos>`가 나올 때까지 다음 단어를 예측하는 행동을 반복한다.

인코더의 입, 출력으로 사용되는 encoder_inputs와 encoder_states는 훈련 과정에서 이미 정의한 것들을 재사용한다. 이렇게 되면 훈련 단계에 encoder_inputs와 encoder_states 사이에 있는 모든 층까지 전부 불러오게 되므로 결과적으로 훈련 단계에서 사용한 인코더를 그대로 재사용하게 된다.  

디코더를 설계할 때는 테스트 단계일 때 디코더를 매 시점 별로 컨트롤 할 예정으로써, 이를 위해서 이전 시점의 상태를 저장할 텐서인 decoder_state_input_h, decoder_state_input_c를 정의한다. 매 시점 별로 디코더를 컨트롤하는 함수는 뒤에서 정의할 decode_sequence()로 해당 함수를 잘 살피는 게 중요하다.

In [28]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 설계
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

# 수정된 디코더
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


아래 코드는 테스트 단계에서의 동작을 위한 decode_sequence 함수를 구현하는 부분이다. 

<코드 설명>
* 입력 문장이 들어오면 인코더는 마지막 시점까지 전개하여 마지막 시점의 은닉 상태와 셀 상태를 리턴한다.  
* 이 두 개의 값을 states_value에 저장한다. 그리고 디코더의 초기 입력으로 `<sos>`를 준비하고 이를 target_seq에 저장한다.  
* 이 두 가지 입력을 가지고 while문 안으로 진입하여 이 두 가지를 디코더의 입력으로 사용한다.  
* 이제 디코더는 현재 시점에 대해서 예측을 하게 될 텐데, 이 때 현재 시점의 예측 벡터로부터 현재 시점의 예측 벡터가 output_tokens, 현재 시점의 은닉 상태를 h, 현재 시점의 셀 상태가 c이다.  
* 예측 벡터로부터 현재 시점의 예측 단어인 target_seq를 얻고, h와 c가 두 개의 값은 states_value에 저장한다.  
* 그리고 while문의 다음 루프 즉, 두번째 시점의 딬더의 입력으로 다시 target_seq와 states_value를 사용한다.  
* 이를 현재 시점의 예측 단어로 `<eos>`를 예측하거나 번역 문장의 길이가 50이 넘는 순간까지 반복한다.  
* 각 시점마다 번역된 단어는 decoded_sentence에 누적하여 저장하였다가 최종 번역 시퀀스로 리턴한다.

In [29]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 정수 생성
  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tar_to_index['<sos>']

  stop_condition = False
  decoded_sentence = ''

  # stop_condition이 True가 될 때까지 루프 반복
  # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 단어로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence += ' '+sampled_char

    # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
    if (sampled_char == '<eos>' or
        len(decoded_sentence) > 50):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

아래 코드는 결과 확인을 위한 함수를 만드는 과정이다.

<코드 설명>
* seq_to_src 함수는 영어 문장에 해당하는 정수 시퀀스를 입력받으면 정수로부터 영어 단어를 리턴하는 index_to_src를 통해 영어 문장으로 변환한다.  
* seq_to_tar은 프랑스어에 해당하는 정수 시퀀스를 입력받으면 정수로부터 프랑스어 단어를 리턴하는 index_to_tar을 통해 프랑스어 문장으로 변환합니다.

In [30]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
      sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence

In [31]:
# 훈련 데이터에 대해 임의로 선택한 인덱스의 샘플의 결과 출력
for seq_index in [7, 45, 400, 700, 2999]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)

입력문장 : i feel frightened . 
정답문장 : je suis effrayee . 
번역문장 : je suis assez satisfait . 
--------------------------------------------------
입력문장 : they re mine . 
정답문장 : elles sont miennes . 
번역문장 : ils sont miennes . 
--------------------------------------------------
입력문장 : are you finished ? 
정답문장 : as tu fini ? 
번역문장 : en avez vous termine ? 
--------------------------------------------------
입력문장 : did you find tom ? 
정답문장 : avez vous trouve tom ? 
번역문장 : avez vous tue tom ? 
--------------------------------------------------
입력문장 : tom is your age . 
정답문장 : tom a votre age . 
번역문장 : tom est une vieille maison . 
--------------------------------------------------


In [33]:
# 테스트 데이터에 대해 임의로 선택한 인덱스의 샘플의 결과 출력
for seq_index in [7, 45, 400, 700, 2999]:
  input_seq = encoder_input_test[seq_index: seq_index +1]
  decoded_sentence = decode_sequence(input_seq)

  print('입력문장 :', seq_to_src(encoder_input_test[seq_index]))
  print('정답문장 :', seq_to_tar(decoder_input_test[seq_index]))
  print('변역문장 :', decoded_sentence[1:-5])
  print("-"*50)

입력문장 : the meat is frozen . 
정답문장 : la viande est congelee . 
변역문장 : le chien est propre . 
--------------------------------------------------
입력문장 : you are lying . 
정답문장 : vous mentez . 
변역문장 : vous etes en train de mentir . 
--------------------------------------------------
입력문장 : can i sit down ? 
정답문장 : puis je m asseoir ? 
변역문장 : puis je m asseoir ? 
--------------------------------------------------
입력문장 : i just feel awful . 
정답문장 : je me sens juste affreusement mal . 
변역문장 : je me suis remarie . 
--------------------------------------------------
입력문장 : you re grounded . 
정답문장 : vous etes terre a terre . 
변역문장 : tu es mal a la . 
--------------------------------------------------


문장이 조금씩 유사한 부분이 보이는 거 같다.

# 회고

이번에는 영어 문장을 단어 단위로 쪼개어 토큰화를 시킨 후 그 문장을 프랑스어로 번역해 주는 번역기를 돌리기 위한 프로젝트를 수행했다. 단어 단위로 쪼갠 번역기를 만들었다고 생각하니 너무 흥미진진했고 성능이 어떻게 나올까 연습할 때 만들었던 번역기보다 향상된 성능을 발휘할까하는 기대감 역시 들었다.

번역기를 돌려보니 성능적으로도 괜찮았다고 생각이 들었다. 훈련 데이터의 경우 91.72%, 테스트 데이터의 경우 86.60% 정도로 나왔으니 말이다.

이번에도 정답이라고 할 수 있는 참고가 매우 될 만한 블로그의 대부분을 인용하였기 때문에 아직까지는 코드 각각이 어떤 식으로 구성되어 있고 이런 것이 미숙할 수도 있지만 지금은 코드가 어떠한 기능이 있는지에 전체적인 이해를 하는 데에 집중키로 했다. 조만간 영한 번역기를 돌려보는 프로젝트나 익스 노드도 하지 않을까 기대를 해 본다.  